In [1]:
import csv
import pandas as pd
import numpy as np
import string
import re
import glob
import os
import operator
import nltk
from nltk.corpus import stopwords
from stemming.porter2 import stem
from nltk.classify.api import ClassifierI

In [2]:
with open("C:/Users/Steven Ma/Desktop/ASU - Business Analytics/Applied Project/FYI_Brands_List.csv", "r") as f:
    d_reader = csv.DictReader(f)
    brand_list = d_reader.fieldnames #get fieldnames from DictReader object and store in list

In [3]:
# Read CSV file and build a dataframe #
path = 'C:/Users/Steven Ma/Desktop/ASU - Business Analytics/Applied Project/Tweets'
csvFiles = glob.glob(path + "/*.csv")
Doc = os.listdir(path)

file_list = []
name_list = []
appended_data = []
test_df = pd.DataFrame()
final_df = pd.DataFrame()

In [4]:
############### User-defined Text_Classifier function ###############
def Text_Classifier(test_df):
    # Manipulating text data #
    #time_list = []
    influencer_tweets = []
    for index, row in test_df.iterrows():
        #time_list.append(row['created_at'])
        tweet_tuple = list([row['id'][2:-1], row['text'][2:], row['Twitter_ID']])
        influencer_tweets.append(tweet_tuple)
    
    # Text processing #
    tweets_test = []
    mouse_list = []
    hashtag_list = []
    sw = stopwords.words("english")
    for (tweet_id, words, twitter_id) in influencer_tweets:
        new_words = words.split()
        for special_word in new_words:
            if special_word.startswith('@'):
                mouse_list.append(special_word)
            elif special_word.startswith('#'):
                hashtag_list.append(special_word)
        words_removed = [''.join(c for c in s if c not in string.punctuation) for s in new_words]
        words_lowered = [e.lower() for e in words_removed]
        for single_word in words_lowered:
            if single_word.startswith('http'):
                words_lowered.remove(single_word)
        words_nonstopped = [w for w in words_lowered if w not in sw]
        words_stemmed = [stem(txt) for txt in words_nonstopped]
        tweets_test.append([tweet_id, words_stemmed, words, twitter_id, mouse_list, hashtag_list, words_nonstopped])
    
    # Add sentiment in the tuple #
    for i in range(len(test_df)):
        tweets_test[i] += list([classifier.classify(extract_features(tweets_test[i][1]))])
    
    # Convert to dataframe #
    lable = ['Tweet_ID', 'Tweets', 'Original_Tweet', 'Twitter_ID', 'At-mark', 'Hashtag', 'Tweet_Token', 'Sentiment']
    tweet_dataframe = pd.DataFrame(tweets_test, columns = lable)
    
    return tweet_dataframe
############### End of function ###############

In [5]:
############### Build a dataframe with TweetID, Tokenized Tweets, TwitterID, Sentiment ###############
for file_name, file in zip(Doc, csvFiles):
    df = pd.read_csv(file, index_col=None, header=0)
    df['Twitter_ID'] = file_name[:-4]
    classified_df = Text_Classifier(df)
    appended_data.append(classified_df)
final_df = pd.concat(appended_data)
############### End ###############

NameError: name 'classifier' is not defined